In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import  OneHotEncoder
from sklearn.model_selection import  train_test_split
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf

In [ ]:
train = pd.read_csv('./../input/train.csv')

In [ ]:
Y = train['label'].values
train.drop(['label'], axis=1, inplace=True)
source_data  = train.values

In [ ]:
ohe = OneHotEncoder()
Y = ohe.fit_transform(Y.reshape(-1,1))

In [ ]:
source_data = source_data.reshape(-1, 28,28)

In [ ]:
plt.figure()
plt.imshow(source_data[123], cmap='gray')
plt.show()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(source_data, Y)

In [ ]:
x = tf.placeholder(shape=[None, 28,28], dtype=tf.float32)
y = tf.placeholder(shape=[None, 10], dtype=tf.float32)

In [ ]:
x

In [ ]:
y

In [ ]:
hidden_size = 32
fw_lstm_cell_1 = tf.contrib.rnn.LSTMCell(
    num_units=hidden_size, 
)

bw_lstm_cell_1 = tf.contrib.rnn.LSTMCell(
    num_units=hidden_size, 
)
outputs, final_fw, final_bw = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(
    [fw_lstm_cell_1],
    [bw_lstm_cell_1],
    x,
    dtype=tf.float32
)

In [ ]:
outputs

In [ ]:
final_fw

In [ ]:
final_bw

In [ ]:
final_bw[0][1], final_fw[0][1]

In [ ]:
final = tf.concat( [final_bw[0][1], final_fw[0][1]], axis=1)
final

In [ ]:
hidden = tf.layers.Dense(units=32, activation=tf.nn.elu)(final)

In [ ]:
hidden

In [ ]:
output = tf.layers.Dense(units=10, activation=None)(hidden)

In [ ]:
output

In [ ]:
prediction = tf.nn.softmax(output)

In [ ]:
prediction

In [ ]:
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output)
)

In [ ]:
loss

In [ ]:
optimizer = tf.train.AdamOptimizer(
    learning_rate=0.0001
)
train_op = optimizer.minimize(loss)

In [ ]:
def iter_data(X, Y, batch_size):
    count_of_datapoints = X.shape[0]
    batch_count = int(count_of_datapoints/batch_size)
    for i in range(batch_count):
        yield X[batch_size*i: batch_size*(1+i)], Y[batch_size*i: batch_size*(1+i)]
    else:
        yield X[batch_size*(1+i):], Y[batch_size*(1+i):]

In [ ]:
epoch=15

In [ ]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    for ep in range(epoch):
        for _tick_id, (batch_x, batch_y) in enumerate(iter_data(X_train, Y_train, batch_size=64)):
            _, _loss = sess.run(
                [train_op, loss],
                feed_dict={
                    x: batch_x,
                    y: batch_y.toarray(),
                }
            )
            if not _tick_id % 50:
                print(_loss)
        
        _y_pred = []
        for _tick_id, (batch_x, batch_y) in enumerate(iter_data(X_test, Y_test, batch_size=64)):
            y_pred = sess.run(
                [prediction],
                feed_dict={
                    x: batch_x,
                }
            )
            _y_pred+=list(y_pred[0].argmax(axis=1))
        print(accuracy_score(Y_test.argmax(axis=1), _y_pred))
        print(classification_report(Y_test.argmax(axis=1), _y_pred)),
        saver.save(sess=sess, save_path='./mnist_lstm.model')
        
